In [ ]:
import sys
import math
from typing import Dict, List
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm')

sys.path.append('..')
from tools.BasicUtils import my_read, my_write, MultiProcessing
from tools.TextProcessing import Occurance, occurance_dump, occurance_load, occurance_post_operation, sent_lemmatize
import networkx as nx

In [ ]:
# Helper functions

# Co-occurance list related functions
def gen_co_occur(occur_dict:Dict[str, set], sent_len:int, word2idx:Dict[str, int]):
    co_occur_list = [set() for i in range(sent_len)]
    for key, set_ in occur_dict.items():
        idx = word2idx[key]
        for line in set_:
            co_occur_list[line].add(idx)
    return co_occur_list

def co_occur_dump(co_occur_file:str, co_occur_list:List[set]):
    my_write(co_occur_file, [' '.join(map(str, set_)) for set_ in co_occur_list])

def co_occur_load(co_occur_file:str):
    return [list(map(int, line.split())) for line in my_read(co_occur_file)]

# NPMI related functions
def build_graph(co_occur_list:List[List[int]], keywords:List[str]):
    g = nx.Graph(c=0)
    g.add_nodes_from(range(len(keywords)), c=0)
    print('Reading Co-occurance lines')
    for line_idx, line in enumerate(co_occur_list):
        kw_num = len(line)
        g.graph['c'] += kw_num * (kw_num - 1)
        for i in range(kw_num):
            u = line[i]
            g.nodes[u]['c'] += (kw_num - 1)
            for j in range(i+1, kw_num):
                v = line[j]
                if not g.has_edge(u, v):
                    g.add_edge(u, v, c=0)
                g.edges[u, v]['c'] += 1
        if line_idx % 5000 == 0:
            print('\r%d' % line_idx, end='')
    print('')
    print('Reading Done! NPMI analysis starts...')
    Z = float(g.graph['c'])
    for e, attr in g.edges.items():
        attr['npmi'] = -math.log((2 * Z * attr['c']) / (g.nodes[e[0]]['c'] * g.nodes[e[1]]['c'])) / math.log(2 * attr['c'] / Z)
    print('NPMI analysis Done')
    return g

def graph_dump(g:nx.Graph, gpickle_file:str):
    nx.write_gpickle(g, gpickle_file)

def graph_load(gpickle_file:str):
    return nx.read_gpickle(gpickle_file)

In [ ]:
# Load fundamental data (50 seconds)
sent_list = my_read('../data/corpus/small_sent_reformed.txt')
keywords = my_read('../data/corpus/entity.txt')
word2idx = {word:i for i, word in enumerate(keywords)}
occur_dict = occurance_load('../data/corpus/ent_occur.json')
co_occur_list = co_occur_load('../data/corpus/ent_co_occur.txt')
pair_graph = graph_load('../data/corpus/ent_pair.gpickle')

In [ ]:
# Generate sentence file with line number
!grep -n '' ../data/corpus/small_sent.txt > ../data/corpus/small_sent_line.txt

In [ ]:
# Generate occurance file
# To run the code in the backend, use the gen_occur.py in the "py" folder
p = MultiProcessing()
occur_dict = p.run(lambda: Occurance('../data/corpus/wordtree.json', '../data/corpus/keyword_f.txt'), open('../data/corpus/small_sent_line.txt').readlines(), 8, occurance_post_operation)
occurance_dump('../data/corpus/occur.json', occur_dict)

In [ ]:
# Remove the file with line number
!rm ../data/corpus/small_sent_line.txt

In [ ]:
# Generate co_occurance file
co_occur_list = gen_co_occur(occur_dict, len(sent_list), word2idx)
co_occur_dump('../data/corpus/ent_co_occur.txt', co_occur_list)

In [ ]:
# Generate pair graph (about 5 minutes)
pair_graph = build_graph(co_occur_list, keywords)
graph_dump(pair_graph, '../data/corpus/ent_pair.gpickle')

Play around in the below

In [ ]:
test_data = my_read('../data/test/co_occur_test.txt')
test_data = [data.split(',') for data in test_data]
test_dict = {data[0] : data[1:] for data in test_data}

In [ ]:
test_sent_dict = {central_kw : set() for central_kw in test_dict}
for central_kw, kws in test_dict.items():
    for kw in kws:
        test_sent_dict[central_kw] |= occur_dict[kw]
    test_sent_dict[central_kw] &= occur_dict[central_kw]

for central_kw, sents in test_sent_dict.items():
    content = [sent_list[i] for i in sents]
    my_write('../data/temp/%s_wiki.txt' % central_kw.replace(' ', '_'), content)

In [ ]:
test_lines = occur_dict['python'] & (occur_dict['java'] | occur_dict['ruby'])
my_write('python_java_ruby.txt', [sent_list[i] for i in test_lines])

Analyze the sentences with OLLIE, Stanford OpenIE or OpenIE5

In [ ]:
openie_data = my_read('../data/temp/pl_wiki_ollie_triple.txt')
# openie_data = my_read('pjr_ollie_triple.txt')
# keywords = set(['data structure', 'binary tree', 'hash table', 'linked list'])
keywords = set(['programming language', 'python', 'java', 'javascript', 'lua', 'scala', 'lisp', 'php', 'ruby', 'smalltalk'])
# keywords = set(['python', 'java', 'ruby'])

qualified_triples = []
for data in openie_data:
    if data:
        arg1, rel, arg2 = data.split(';')
        for kw in keywords:
            if kw in arg1:
                for kw in keywords:
                    if kw in arg2:
                        qualified_triples.append(data)
                        break
                break
my_write('pl_wiki_ollie_triple_f.txt', qualified_triples)

In [ ]:
data_structure_idx = occur_dict['data structure']

In [ ]:
co_occur_set = {}
for keyword, idx_set in occur_dict.items():
    intersection = idx_set & data_structure_idx
    if intersection:
        co_occur_set[keyword] = list(intersection)

In [ ]:
len(co_occur_set)

In [ ]:
sorted_co_occur_list = sorted(co_occur_set.items(), key=lambda x: len(x[1]), reverse=True)[:100]
sorted_co_occur_count = [(word, len(idx)) for word, idx in sorted_co_occur_list]

In [ ]:
sorted_co_occur_count[:40]

In [ ]:
'b-tree' in co_occur_set

In [ ]:
# sent_list[co_occur_set['b-tree'][0]]
temp_list = [sent_list[idx] for idx in co_occur_set['b-tree']]
my_write('ds_bt_sent.txt', temp_list)

Test of highly related pairs

In [ ]:
# Helper functions
def get_subgraph(g:nx.Graph, npmi_threshold:float, min_count:int):
    return g.edge_subgraph([e[0] for e in pair_graph.edges.items() if e[1]['npmi'] > npmi_threshold and e[1]['c'] >= min_count])

def find_dependency_path(sent:str, kw1:str, kw2:str):
    doc = nlp(sent)
    tokens = [t.text for t in doc]
    try:
        idx1 = tokens.index(kw1)
        idx2 = tokens.index(kw2)
    except:
        return ''
    branch = np.zeros(len(doc))
    i = idx1
    while branch[i] == 0:
        branch[i] = 1
        i = doc[i].head.i
    i = idx2
    while branch[i] == 0:
        branch[i] = 2
        i = doc[i].head.i
    dep1 = []
    j = idx1
    while j != i:
        dep1.append('i_%s' % doc[j].dep_)
        j = doc[j].head.i
    dep2 = []
    j = idx2
    while j != i:
        dep2.append(doc[j].dep_)
        j = doc[j].head.i
    dep2.reverse()
    if branch[idx2] == 1:
        # kw2 is along the heads of kw1
        return ' '.join(dep1)
    elif i == idx1:
        # kw1 is along the heads of kw2
        return ' '.join(dep2)
    else:
        return ' '.join(dep1 + dep2)

def mark_sent_in_html(sent:str, keywords:List[str], is_entity:bool=True):
    reformed_sent = sent.split() if is_entity else sent_lemmatize(sent.replace('-', ' - '))
    reformed_keywords = [[k] for k in keywords] if is_entity else [k.replace('-', ' - ').split() for k in keywords]
    mask = np.zeros(len(reformed_sent), dtype=np.bool)
    for k in reformed_keywords:
        begin_idx = 0
        while reformed_sent[begin_idx:].count(k[0]) > 0:
            begin_idx = reformed_sent.index(k[0], begin_idx)
            is_good = True
            i = 0
            for i in range(1, len(k)):
                if begin_idx + i >= len(reformed_sent) or reformed_sent[begin_idx + i] != k[i]:
                    is_good = False
                    break
            if is_good:
                mask[begin_idx:begin_idx+i+1] = True
            begin_idx += (i+1)
    i = 0
    insert_idx = 0
    while i < len(mask):
        if mask[i] and (i == 0 or mask[i-1] == False):
            reformed_sent.insert(insert_idx, '<font style=\"color:red;\">')
            insert_idx += 2
            i += 1
            while i < len(mask) and mask[i]:
                i += 1
                insert_idx += 1
            reformed_sent.insert(insert_idx, '</font>')
            insert_idx += 1
        insert_idx += 1
        i += 1
    return ' '.join(reformed_sent)

def gen_co_occur_report(report_file:str, g:nx.Graph, keyword:str, word2idx:Dict[str, int], keyword_list:List[str], occur_dict:Dict[str, set], sent_list:List[str], is_entity:bool=True, kw_dist_max:int=6):
    neighbors = g.neighbors(word2idx[keyword])
    related_kws = [keywords[idx] for idx in neighbors]
    content = ['<a href=\"#%s__%s\">%s, %s</a><br>' % (keyword, kw, keyword, kw) for kw in related_kws]
    for kw in related_kws:
        content.append('<a id=\"%s__%s\"><h1>%s, %s</h1></a> ' % (keyword, kw, keyword, kw))
        sents = [sent_list[i] for i in occur_dict[keyword] & occur_dict[kw]]
        if is_entity:
            sents = [sent.split() for sent in sents]
            sents = [' '.join(sent) for sent in sents if sent.count(keyword) == 1 and sent.count(kw) == 1 and abs(sent.index(keyword) - sent.index(kw)) <= kw_dist_max]
        content += ['%s<br><br>' % mark_sent_in_html(sent, [keyword, kw], is_entity=is_entity) for sent in sents]
    
    my_write(report_file, content)

In [ ]:
# Generate highly related subgraph
sub_g = get_subgraph(pair_graph, 0.3, 3)

In [ ]:
neighbors = sub_g.neighbors(word2idx['data_structure'])
related_kws = [keywords[idx] for idx in neighbors]
print(related_kws)

In [ ]:
sents = [sent_list[i] for i in (occur_dict['python'] & occur_dict['just-in-time compilation'])]
for sent in sents:
    print(sent)

In [ ]:
sub_g.edges[word2idx['python'], word2idx['just-in-time compilation']]

In [ ]:
gen_co_occur_report('python_co_occur.html', sub_g, 'python', word2idx, keywords, occur_dict, sent_list, 6)